In [ ]:
#3.11 version
from dotenv import load_dotenv
load_dotenv() # loading the env key
from langchain_groq import ChatGroq
import os
GROK_API_KEY = os.getenv("GROK_API_KEY")

llm = ChatGroq(
    temperature = 0,
    groq_api_key = GROK_API_KEY,
    model_name = 'llama-3.1-70b-versatile')

response = llm.invoke("India won world cup on 2017 ..")
print(response.content)

In [ ]:
GROK_API_KEY = os.getenv("GROK_API_KEY")
print(GROK_API_KEY)

In [ ]:
# !pip install langchain_community
# !pip install langchain_groq
# !pip install chromadb
# pip show langchain
# pip install --upgrade langchain
# pip show sqlalchemy
# pip install --upgrade sqlalchemy
# pip install --upgrade langchain_community
# pip show langchain_community


In [ ]:
import langchain 
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jobs.nike.com/job/R-41251")
page_data = loader.load().pop().page_content
print(page_data)

#### In Chat-GPT I will give extracted job posting. can you extract the skills, roles and description

 Output from chat-gpt -- Skills:

    Proficiency in Python for developing robust, maintainable, and extendable code
    Strong background in algorithms and data structures
    AWS (Amazon Web Services) hands-on experience
    Database technology knowledge, especially with Postgres and Redis
    Data processing tools like EMR
    Experience in containerization (Docker) and CI/CD automation
    Agile and test-driven development expertise
    Data engineering skills, including working with ETL pipelines and SQL
    MLOps, API development, and mathematical optimization
    Familiarity with Spark, Kubernetes, Jenkins, Databricks, and Terraform
    Effective communication skills

Role:

Senior Machine Learning Engineer

Responsibilities include:

    Developing advanced analytics and machine learning solutions
    Designing and implementing scalable applications for data-driven decision-making
    Contributing to advanced analytics and ML platforms
    Leading projects end-to-end with a focus on the software development lifecycle
    Providing technical vision and guidance to the team
    Collaborating with global teams in a diverse, inclusive environment

Job Description:

As a Senior ML Engineer, you’ll be part of Nike’s AI/ML team, working to solve large-scale machine learning challenges. This role involves creating prediction models and optimization programs, contributing to both analytics and ML platforms, and working with a global team across multiple locations. The position values a collaborative, intellectually curious culture that supports continuous learning and knowledge-sharing within Nike and the broader community.
Additional Information:

    Location: Beaverton, Oregon (Open to remote, with restrictions for certain states)
    Salary range: $99,500 - $222,900

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    ### INSTRUCTION
    The scraped text from the career's page of a website.
    Your job is to extract the job posting and return them in JSON format containing the
    following keys: 'role', 'experience', 'skills' and 'description'.
    Only return the valid JSON.
    ### VALIDA JSON (No presemble):
    """
)

chain_extract = prompt_extract | llm #This will form chain
response = chain_extract.invoke({"page_data": page_data})
print(response.content)
# its power of llm - it can extract key information among the all the clutter

In [ ]:
# Converting str into Json
from langchain_core.output_parsers import JsonOutputParser
json_parser = JsonOutputParser()
json_res = json_parser.parse(response.content)
print(json_res)

In [ ]:
print(type(json_res))

In [ ]:
# Reading CSV for upload a portfolio
import pandas as pd
df = pd.read_csv('my_portfolio.csv')
df

In [ ]:
# Iterating the dataframe one by one and add information to chromadb
import uuid
import chromadb
client = chromadb.PersistentClient('vectorstore')
#When use persistent client, it will create a folder called vector store vectore details
collection = client.get_or_create_collection(name='my_portfolio')

if not collection.count():
  for _, row in df.iterrows():
    collection.add(documents=row['Techstack'],
                    metadatas = {"links":row["Links"]},
                    ids = [str(uuid.uuid4())])

In [ ]:
links = collection.query(query_texts = ['Experience in python ', "Expertise in React Native"], n_results=2).get("metadata",[])
links

In [ ]:
collection

In [ ]:
job = json_res
job['skills']

In [ ]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

In [ ]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION:
    {job_description}

    ### INSTRUCTION:
    You are Sivaranjani, a business development at xxx company, xxx company is an AI & Software consulting company dedicated to the seamless integration
    of business processes through automated tools.
    Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalablity, process optimization, cost reduction,
    and heightened overall efficiency.
    Your job is to write a cold email to the client regarding the job mentioned above describing the capablity of xxx in fulfilled their needs.
    Also add the most relevent ones from the following links to showcase xxx's portfolio : {link_list}
    Remember you are Sivaranjani, BDE at xxx.
    Do not provide a preambel.
    ### EMAIL (NO PREAMBLE):
    """
)

In [ ]:
chain_email = prompt_email | llm
res = chain_email.invoke({'job_description' : str(job), "link_list":links })
print(res.content)